# Pandas

파이썬에서 가장 널리 사용되는 데이터 분석 및 조작 라이브러리

다음과 같은 핵심 데이터 구조를 가지고 있음

- DataFrame : 2차원 테이블 형태의 데이터 구조 (엑셀 시트와 유사)
- Series : 1차원 배열 형태의 데이터 구조

Attribute인지 Method인지, 반환값이 있는지 없는지를 추가로 확인해 보는것을 추천함


# Series

1D Labeled data structure로 하나의 row 또는 하나의 column을 추상화 하고 있는 Class임

pandas의 1D data를 위한 핵심 데이터 구조

index와 value로 구성된 labeled 1차원 데이터 구조를 관리함

- index : 각 데이터 포인트의 레이블 (unique할 필요 없음)

  - 흔히, 0부터 시작하는 정수 index로 접근 : .iloc
  - 또는 명시적으로 할당된 label을 통해 접근 : .loc

- value : 실제 데이터 값
  - 모든 값은 동일한 데이터 타입(dtype)을 가짐
  - 선택적으로 Series는 name속성을 가질 수 있음 (Series를 가르키는 이름)

DataFrame은 2D Labeled Tabular data structure로, 여러 개의 Series가 열(column)로 결합된 형태라고 볼 수 있음. 즉, DataFrame의 각 column은 Series임

## 1. Series 생성

### 1-1. list로 생성


In [2]:
import pandas as pd
import numpy as np

# pandas는 내부적으로 numpy array 를 기반으로 동작함
s1 = pd.Series([10, 20, 30, 40, 50])
print("리스트로 생성")
print(s1)

리스트로 생성
0    10
1    20
2    30
3    40
4    50
dtype: int64


| 구성 요소                   | 설명                                                                       |
| --------------------------- | -------------------------------------------------------------------------- |
| Index                       | 행/열의 라벨 (numpy에는 없음, pandas에서 추가된 개념)                      |
| Data                        | 실제 값 저장. numpy.ndarray 또는 ExtensionArray 형태                       |
| dtype                       | NumPy dtype(예: int64, float64 등) 혹은 pandas 전용 dtype                  |
| BlockManager / ArrayManager | DataFrame 내부에서 열 단위 데이터를 효율적으로 관리하는 pandas 내부 구조체 |

### 1-2. index를 지정하여 생성


In [3]:
s2 = pd.Series([83, 92, 98],
               index=['김철수', '김영희', '김행근'],
               name='점수')

print(s2)

김철수    83
김영희    92
김행근    98
Name: 점수, dtype: int64


### 1-3. dictionary로 생성


In [4]:
dict_data = {'서울': 9765, '부산': 3419, '인천': 2958, '대구': 2427}
s3 = pd.Series(dict_data, name='인구(천명)')

print(s3)

서울    9765
부산    3419
인천    2958
대구    2427
Name: 인구(천명), dtype: int64


## 2.Series의 Attribute

Series는 Class이기 때문에 속성값을 가질 수 있음

Attritube는 객체의 상태를 나타내는 데이터 값으로 Method와 다르게 실행 없이 값을 반환함


In [5]:
print(f"values : {s2.values}")
print(f"index : {s2.index}")
print(f"name : {s2.name}")
print(f"size : {s2.size}")
print(f"dtype : {s2.dtype}")

values : [83 92 98]
index : Index(['김철수', '김영희', '김행근'], dtype='object')
name : 점수
size : 3
dtype : int64


- `.values` 보다는 `.to_numpy(dtype='float64', copy='True')`가 권장됨
  - numpy array로 반환 되어 용이함
- 값을 읽는 경우만 `.values` 또는 `.to_numpy()`를 사용
- 값을 바꿔야 하는 경우에는 `indexer`를 사용하거나 `filtering` 또는 `apply`를 사용


In [6]:
s2.to_numpy(dtype='float64', copy='True')

array([83., 92., 98.])

In [7]:
print(type(s2.values))  # numpy array
print(type(s2.index))  # Index type
print(type(s2.name))  # str
print(type(s2.size))  # int
print(type(s2.dtype))  # numpy dtype

<class 'numpy.ndarray'>
<class 'pandas.core.indexes.base.Index'>
<class 'str'>
<class 'int'>
<class 'numpy.dtypes.Int64DType'>


## 3. data 접근

panda에서는 `loc`, `iloc`, `at`, `iat`의 indexer를 제공함


### 3-1. index로 접근 (`.iloc`)


In [8]:
s2['김철수']  # indexer .loc을 사용하는 경우와 동일하게 작동

np.int64(83)

In [9]:
s2.loc['김철수']  # NumPy 스칼라 값이 반환

np.int64(83)

In [10]:
s2.loc[['김행근']]  # 부분 시리즈 반환

김행근    98
Name: 점수, dtype: int64

In [11]:
s2.loc[['김철수', '김영희']]  # 부분 시리즈

김철수    83
김영희    92
Name: 점수, dtype: int64

### 3-2. 위치로 접근 (`.loc`)


In [12]:
s2.iloc[0:2]  # 부분 시리즈 반환

김철수    83
김영희    92
Name: 점수, dtype: int64

In [13]:
s2.iloc[0]  # NumPy 스칼라 값이 반환됨환

np.int64(83)

## 4. 데이터 필터링

condition에 의해 boolean mask(Series 객체)가 생성되며 이를 이용한 indexing임

- 여러개의 condition을 사용시 각각을 ()로 감싸야 함.
- &, |, ~ 를 사용하여 condition을 묶음 (and, or, not 사용 불가)


In [14]:
data_list = [10, 20, 30, 40, 50]

s3 = pd.Series(data_list, index=['정은교', '배서준',
               '이원준', '최일환', '이제석'], name='score')

In [15]:
print('students who have a score over 30')
print(s3[s3 >= 30])

students who have a score over 30
이원준    30
최일환    40
이제석    50
Name: score, dtype: int64


In [16]:
s3 >= 30

정은교    False
배서준    False
이원준     True
최일환     True
이제석     True
Name: score, dtype: bool

In [17]:
print('students who have a score between 20 and 30')
print(s3[(s3 >= 20) & (s3 <= 30)])

students who have a score between 20 and 30
배서준    20
이원준    30
Name: score, dtype: int64


# DataFrame

pandas의 핵심 데이터 구조

엑셀 시트처럼 행과 열로 구성된 labeled 2차원 tavular data를 관리

- Row : a case of sample (=single instance)
  - 흔히, 0부터 시작하는 index를 통해 접근 : .iloc
  - 또는 index로 할당된 label을 통해 접근 : .loc
- Column : a feature (or attribute).
  - DataFrame 에서 각각의 Column은 문자열 이름을 가진 Series라고 볼 수 있음

> 참고사항 0 :
>
> pandas 2.0부터는 \
> DataFrame.append()와 Series.append() 메서드가 완전히 제거(deprecated)되어 \
> pd.concat()을 이용한 결합 방식이 표준이 됨.
>
> 참고사항 1 :
>
> 문자열 데이터(Python의 str)는 기본적으로 object dtype으로 생성됨: \
> 결측값 처리와 타입 일관성을 위해서는 \
> dtype="string" 으로 명시적으로 지정하는 것이 권장됨. \
> 이는 Pandas 전용 nullable string타입으로 Python의 str과 다름.


## Column을 직접 할당하여 DataFrame 생성

새로운 empty DataFrame 객체를 만든 뒤, 각 Column에

- list,
- numpy array
- Series 등을 직접 할당하여

DataFrame 객체를 만드는 방식

문자열은 기본적으로 `object dtype`이지만 결측치(pd.NA)와 처리와 타입 일관성을 위해 `dtype="string"`을 명히적으로 지정하는 것이 권장됨

#### 이게 뭔 소리냐면..

pandas에선 기본적으로 `object dtype`을 할당함, `object dtype`은 모든 타입을 가질 수 있고 `NaN(float형 결측치)`와 문자열이 섞여 있을 수 있어 타입 일관성이 깨질 수 있음

이 때 `dtype="string"`으로 설정하면 `StringDtype`으로 설정되어 결측치를 NA로 처리히게됨


In [18]:
import pandas as pd
import numpy as np

df = pd.DataFrame()

# 문자열 열: dtype="string"으로 명시 (nullable)
df["Name"] = pd.Series(["Alice", "Bob", "Charlie"], dtype="string")

# 정수형 열: nullable 정수 dtype (Int64) 사용
df["Age"] = pd.Series([25, 30, 35], dtype="Int64")

# 실수형 열: NumPy 배열로 할당
df["Score"] = np.array([88.5, 92.0, 79.0])

# Series 할당 시 인덱스 레이블이 겹치는 위치에만 값이 채워짐
s_city = pd.Series(["Seoul", "Busan"], index=[0, 2],
                   name="City", dtype="string")
df["City"] = s_city

print(df)

      Name  Age  Score   City
0    Alice   25   88.5  Seoul
1      Bob   30   92.0   <NA>
2  Charlie   35   79.0  Busan


## 개별 Series를 기존 DataFrame에 새로운 Column으로 추가하기

`Series` 객체를 기존의 `DataFrame`에 새로운 `column`으로 합칠 때 (추가)는,\
해당 `Series` 객체에서 .`to_frame()` 메서드를 통해 `DataFrame` 객체로 변환한 뒤\
`pd.concat(axis=1)`으로 결합(`axis=1` 이어야 column으로 결합됨)시킬 수 있음.

- pandas는 인덱스 레이블을 기준으로 정합(label-based alignment)해 결합함.
- `pd.concat()`함수의 기본 동작은 `outer join` 이나, join파라미터에 `"inner"`를 할당해서 `inner join` 모드로 동작 가능함.

  - `pd.concat()`함수는 `left join`과 `right join`은 지원 안함.

  - 이는 `DataFrame`의 메서드인 `join()`,`merge()`에서 지원.


In [19]:
import pandas as pd

# 기준 DataFrame
base = pd.DataFrame({"Name": pd.Series(["Alice", "Bob", "Charlie"], dtype="string",
                    index=[100, 101, 102])})

# 각각 일부 인덱스를 가진 Series
age_s = pd.Series([25, 30], index=[100, 101], name="Age", dtype="Int64")
city_s = pd.Series(["Seoul", "Busan", "Incheon"], index=[100, 102, 103],
                   name="City", dtype="string")

df.index = df.index * 10

# Series를 DataFrame으로 변환
df = pd.concat([base, age_s.to_frame(), city_s.to_frame()],
               axis=1)  # default outer join
print(df)

        Name   Age     City
100    Alice    25    Seoul
101      Bob    30     <NA>
102  Charlie  <NA>    Busan
103     <NA>  <NA>  Incheon


# Basic Attribute and Exploration Methods

pandas의 DataFrame객체는 2차원 데이터 구조(2D tabular structure)로, 데이터 분석에서 가장 자주 사용되는 객체임.

- 일반적으로 데이터에서 수백 ~ 수십만의 row (case) 및 column (feature, attribute)이 존재
- 일부 데이터를 출력하거나 통계치로서 데이터를 살펴보는 과정 필요. ← Descriptive Statistics

이같은 DataFrame 객체의 구조 및 내용을 빠르게 파악하기 위한 주요 attributes와 exploration methods를 소개한다.


### 1. DataFrame 기본 속성 (Attributes)

- DataFrame 객체는 NumPy 배열처럼 몇 가지 기초 속성을 바로 확인할 수 있음
- shape, ndim, dtype 등을 손쉽게 확인 가능

*다음중 Attribute 인걸 골라라

In [20]:
# import pandas as pd

# # 예제 DataFrame
# data = {
#     "Name": ["Alice", "Bob", "Charlie", "David"],
#     "Age": [25, 30, 35, 40],
#     "City": ["Seoul", "Busan", "Incheon", "Daegu"]
# }
# df = pd.DataFrame(data, dtype="string")

# 주요 속성 확인
print("Shape:", df.shape)        # (행, 열)
print("ndim:", df.ndim)          # 차원 수 (항상 2)
print("Size:", df.size)          # 전체 원소 개수
print("dtypes:\n", df.dtypes)    # 각 column의 dtype
print("Index:", df.index)        # 행 인덱스 객체
print("Columns:", df.columns)    # 열 이름

Shape: (4, 3)
ndim: 2
Size: 12
dtypes:
 Name    string[python]
Age              Int64
City    string[python]
dtype: object
Index: Index([100, 101, 102, 103], dtype='int64')
Columns: Index(['Name', 'Age', 'City'], dtype='object')


### 2. 구조 확인 메서드: info()

- 데이터의 전반적 구조 요약 제공
- 행 개수, 열 개수, 각 열의 데이터 타입, 결측치 여부 등을 확인 가능
  

In [21]:
df.info() # return None

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 100 to 103
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3 non-null      string
 1   Age     2 non-null      Int64 
 2   City    3 non-null      string
dtypes: Int64(1), string(2)
memory usage: 132.0 bytes


In [22]:
a = df.info()

print("-"*10, a, sep='\n')

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 100 to 103
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3 non-null      string
 1   Age     2 non-null      Int64 
 2   City    3 non-null      string
dtypes: Int64(1), string(2)
memory usage: 132.0 bytes
----------
None


### 3. 통계 요약 메서드 : describe()

In [23]:
print(df.describe()) #return DataFrame

            Age
count       2.0
mean       27.5
std    3.535534
min        25.0
25%       26.25
50%        27.5
75%       28.75
max        30.0


In [24]:
a = df.describe()

In [25]:
type(a)

pandas.core.frame.DataFrame

In [26]:
a

,Age
count,2.0
mean,27.5
std,3.535534
min,25.0
25%,26.25
50%,27.5
75%,28.75
max,30.0


In [28]:
for idx, i in df.iterrows():
  print(idx, " ")
  print(type(i))
  print(i)
  print("-"*10)

100  
<class 'pandas.core.series.Series'>
Name    Alice
Age        25
City    Seoul
Name: 100, dtype: object
----------
101  
<class 'pandas.core.series.Series'>
Name     Bob
Age       30
City    <NA>
Name: 101, dtype: object
----------
102  
<class 'pandas.core.series.Series'>
Name    Charlie
Age        <NA>
City      Busan
Name: 102, dtype: object
----------
103  
<class 'pandas.core.series.Series'>
Name       <NA>
Age        <NA>
City    Incheon
Name: 103, dtype: object
----------


In [29]:
df.columns

Index(['Name', 'Age', 'City'], dtype='object')

In [30]:
df.index

Index([100, 101, 102, 103], dtype='int64')

In [31]:
df.columns = ['이름', '나이', '도시']

In [32]:
df.index = [str(x) for x in range(10, 14)]

In [33]:
df

,이름,나이,도시
10,Alice,25,Seoul
11,Bob,30,<NA>
12,Charlie,<NA>,Busan
13,<NA>,<NA>,Incheon


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 10 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      3 non-null      string
 1   나이      2 non-null      Int64 
 2   도시      3 non-null      string
dtypes: Int64(1), string(2)
memory usage: 132.0+ bytes


In [36]:
df.set_index("이름")

,나이,도시
이름,,
Alice,25,Seoul
Bob,30,<NA>
Charlie,<NA>,Busan
<NA>,<NA>,Incheon


In [37]:
df2 = df.set_index("이름")

In [39]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, Alice to <NA>
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   나이      2 non-null      Int64 
 1   도시      3 non-null      string
dtypes: Int64(1), string(1)
memory usage: 100.0+ bytes


In [40]:
df2.reset_index()

,이름,나이,도시
0,Alice,25,Seoul
1,Bob,30,<NA>
2,Charlie,<NA>,Busan
3,<NA>,<NA>,Incheon


In [41]:
df3 = df2.reset_index(drop=True)

In [42]:
df3

,나이,도시
0,25,Seoul
1,30,<NA>
2,<NA>,Busan
3,<NA>,Incheon


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 10 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이름      3 non-null      string
 1   나이      2 non-null      Int64 
 2   도시      3 non-null      string
dtypes: Int64(1), string(2)
memory usage: 132.0+ bytes


In [5]:
import pandas as pd

df = pd.DataFrame({"name" : ["a", "b"], "age" : [10, 20]})

for _, i in df.iterrows():
  print(i)

name     a
age     10
Name: 0, dtype: object
name     b
age     20
Name: 1, dtype: object


In [2]:
[ n * 2 for n in range(2, 10)]

[4, 6, 8, 10, 12, 14, 16, 18]

In [1]:
import pandas as pd

# 데이터프레임 생성
data = {'Name': ['Anna', 'Bob', 'Charlie'],
        'Age': [24, None, 30],
        'City': ['New York', 'Los Angeles', None]}
df = pd.DataFrame(data)

a = df.dropna(subset=['City'])
b = df.dropna()

In [2]:
a

,Name,Age,City
0,Anna,24.0,New York
1,Bob,NaN,Los Angeles


In [3]:
b

,Name,Age,City
0,Anna,24.0,New York


In [4]:
import pandas as pd

# DataFrame 생성
data = {
    "Name": ["Alice", "Bob", "Charlie", "Alice", "David", "Bob"],
    "Gender": ["Female", "Male", "Male", "Female", "Male", "Male"],
    "Class": ["1st", "2nd", "3rd", "1st", "*", "2nd"]
}

df = pd.DataFrame(data)
print(df)

      Name  Gender Class
0    Alice  Female   1st
1      Bob    Male   2nd
2  Charlie    Male   3rd
3    Alice  Female   1st
4    David    Male     *
5      Bob    Male   2nd


In [5]:
df["Gender"].unique()

# 출력:
# 
# array(['Female', 'Male'], dtype=object)

array(['Female', 'Male'], dtype=object)

In [6]:
df["Gender"].value_counts()

Gender
Male      4
Female    2
Name: count, dtype: int64

In [7]:
df["Gender"].nunique()

2

In [4]:
import pandas as pd

# DataFrame 생성
data = {
    "Name": ["Alice", "Bob", "Charlie", "Alice", "David", "Bob"],
    "Gender": ["Female", "Male", "Male", "Female", "Male", "Male"],
    "Class": ["1st", "2nd", "3rd", "1st", "*", "2nd"]
}

df = pd.DataFrame(data)
print(df)

      Name  Gender Class
0    Alice  Female   1st
1      Bob    Male   2nd
2  Charlie    Male   3rd
3    Alice  Female   1st
4    David    Male     *
5      Bob    Male   2nd


In [5]:
df['Gender'].unique()


array(['Female', 'Male'], dtype=object)

In [6]:
df['Gender'].value_counts()

Gender
Male      4
Female    2
Name: count, dtype: int64

In [7]:
df['Gender'].nunique()

2

In [8]:
df['Gender'].isnull()

0    False
1    False
2    False
3    False
4    False
5    False
Name: Gender, dtype: bool

In [9]:
df['Gender'].notnull()

0    True
1    True
2    True
3    True
4    True
5    True
Name: Gender, dtype: bool

In [10]:
df['Gender'].dropna()

0    Female
1      Male
2      Male
3    Female
4      Male
5      Male
Name: Gender, dtype: object

In [24]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    {
        "Age":  [22, 38, 26, 35, np.nan],
        "Fare": [7.25, 71.2833, 7.925, 53.1, 8.05],
        "VIP":  [False, True, False, True, False],   # boolean:  0/1 집계
        "Name": ["Allen", "Baker", "Cody", "Dawn", "Evan"],  # 문자열(object)
    }
)

s_age = df["Age"]

df

,Age,Fare,VIP,Name
0,22.0,7.2500,False,Allen
1,38.0,71.2833,True,Baker
2,26.0,7.9250,False,Cody
3,35.0,53.1000,True,Dawn
4,NaN,8.0500,False,Evan


In [22]:
print(df["Age"].sum(), end='\n\n')

print(df.sum(numeric_only=True), end='\n\n')

print(df.sum(axis=1, numeric_only=True), end='\n\n')

121.0

Age     121.0000
Fare    147.6083
VIP       2.0000
dtype: float64

0       29.25
1    110.2833
2      33.925
3        89.1
4        8.05
dtype: object



In [18]:
print(df["Age"].count())

4


In [19]:
print(df.count())

Age     4
Fare    5
VIP     5
Name    5
dtype: int64


In [31]:
print(df["Age"].max(), df["Age"].min())     # Age 열 최댓값/최솟값
print(df.max(numeric_only=True))            # DataFrame 열별 최댓값
print(df.min(axis=1, numeric_only=True))    # 행별 최솟값


38.0 22.0
Age        38.0
Fare    71.2833
VIP        True
dtype: object
0    False
1     True
2    False
3     True
4    False
dtype: object


In [32]:
print(df["Age"].mean())                     # Age 열 평균
print(df.mean(numeric_only=True))           # DataFrame 열별 평균

30.25
Age     30.25000
Fare    29.52166
VIP      0.40000
dtype: float64


In [25]:
print(df["Age"].max() - df["Age"].min())  # Age 열 범위
print(df[["Age","Fare"]].apply(lambda s: s.max() - s.min()))  # 열별 범위

16.0
Age     16.0000
Fare    64.0333
dtype: float64


In [33]:
print(df["Age"].median())                   # Age 열 중앙값
print(df.median(numeric_only=True))         # DataFrame 열별 중앙값

30.5
Age     30.50
Fare     8.05
VIP      0.00
dtype: float64


In [34]:
print(df["Age"].mode())                     # Age 열 최빈값
print(df[["Age","Fare"]].mode(numeric_only=True))  # 여러 열의 최빈값

0    22.0
1    26.0
2    35.0
3    38.0
Name: Age, dtype: float64
    Age     Fare
0  22.0   7.2500
1  26.0   7.9250
2  35.0   8.0500
3  38.0  53.1000
4   NaN  71.2833


In [35]:
print(df["Age"].var(ddof=1), df["Age"].std(ddof=1))   # 표본 분산/표준편차
print(df.var(numeric_only=True))                      # DataFrame 열별 분산
print(df.std(axis=1, ddof=0, numeric_only=True))      # 행별 모집단 표준편차

56.25 7.5
Age      56.25000
Fare    930.86186
VIP       0.30000
dtype: float64
0     9.153779
1    28.706407
2    10.880723
3    21.597376
4        4.025
dtype: object


In [36]:
print(df["Age"].skew())                   # Age 열 왜도
print(df.skew(numeric_only=True))         # DataFrame 열별 왜도

-0.1037037037037037
Age    -0.103704
Fare    0.805768
VIP     0.608581
dtype: float64


In [37]:
print(df["Age"].kurt())                   # Age 열 첨도
print(df.kurt(numeric_only=True))         # DataFrame 열별 첨도

-3.93916049382716
Age    -3.939160
Fare   -2.190936
VIP    -3.333333
dtype: float64


In [38]:
print(df["Age"].sem())                    # Age 열 SEM
print(df.sem(numeric_only=True))          # DataFrame 열별 SEM

3.75
Age      3.750000
Fare    13.644500
VIP      0.244949
dtype: float64


In [40]:
print(df["Age"].quantile(0.75))           # Age 열 3사분위수
print(df.quantile([0.25,0.5,0.75], numeric_only=True))  # DataFrame 열별 사분위수

35.75


TypeError: numpy boolean subtract, the `-` operator, is not supported, use the bitwise_xor, the `^` operator, or the logical_xor function instead.

In [41]:
print(df["Age"].max() - df["Age"].min())  # Age 열 범위
print(df[["Age","Fare"]].apply(lambda s: s.max() - s.min()))  # 열별 범위


16.0
Age     16.0000
Fare    64.0333
dtype: float64


In [ ]:
df.agg(
    Age_mean=("Age","mean"),
    Fare_q75=("Fare", lambda s: s.quantile(.75))
)

,Age,Fare
Age_mean,30.25,NaN
Fare_q75,NaN,53.1


In [28]:

df.groupby("VIP", dropna=False).agg(
    Age_mean=("Age","mean"),
    Fare_mean=("Fare","mean"),
    Age_count=("Age","count")
)

,Age_mean,Fare_mean,Age_count
VIP,,,
False,24.0,7.741667,2
True,36.5,62.191650,2


In [29]:
print(df.corr(numeric_only=True)) # Pearson Colleration
#            Age      Fare       VIP
# Age   1.000000  0.209126  0.904534
# Fare  0.209126  1.000000  0.348876
# VIP   0.904534  0.348876  1.000000

print(df.corr(method="spearman", numeric_only=True))   # Spearman 순위 상관
print(df.corr(method="kendall", numeric_only=True))    # Kendall tau 상관
 

          Age      Fare       VIP
Age   1.00000  0.975880  0.962250
Fare  0.97588  1.000000  0.977498
VIP   0.96225  0.977498  1.000000
           Age      Fare       VIP
Age   1.000000  1.000000  0.894427
Fare  1.000000  1.000000  0.866025
VIP   0.894427  0.866025  1.000000
           Age      Fare       VIP
Age   1.000000  1.000000  0.816497
Fare  1.000000  1.000000  0.774597
VIP   0.816497  0.774597  1.000000


In [ ]:
print(df.cov(numeric_only=True))

             Age        Fare        VIP
Age    56.250000  237.058942   4.166667
Fare  237.058942  930.861860  16.334995
VIP     4.166667   16.334995   0.300000


In [45]:
import pandas as pd

# 1) 기본 예제 데이터
df = pd.DataFrame({
    "class": ["A","A","A","B","B","C","C","C","C"],
    "sex":   ["F","M","F","M","F","M","F","F","M"],
    "score": [80, 70, 90, 60, 75, 88, 92, 85, 73],
    "age":   [20, 22, 21, 24, 23, 26, 22, 21, 27],
})

df

,class,sex,score,age
0,A,F,80,20
1,A,M,70,22
2,A,F,90,21
3,B,M,60,24
4,B,F,75,23
5,C,M,88,26
6,C,F,92,22
7,C,F,85,21
8,C,M,73,27


In [46]:
df.groupby("class").mean(numeric_only=True)

,score,age
class,,
A,80.0,21.0
B,67.5,23.5
C,84.5,24.0


In [51]:
df.groupby(["class","sex"])["score"].mean()

class  sex
A      F      85.0
       M      70.0
B      F      75.0
       M      60.0
C      F      88.5
       M      80.5
Name: score, dtype: float64

In [49]:
row_groups = [i % 2 for i in range(len(df))]
df.groupby(row_groups)["score"].mean()

0    82.00
1    75.75
Name: score, dtype: float64

In [52]:
df_idx = pd.DataFrame({"val":[10,20,30]}, index=["row1","row2","row3"])
map_idx = {"row1":"A","row2":"A","row3":"B"}
print(df_idx.groupby(map_idx, axis=0).sum())

   val
A   30
B   30


/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_93472/3658089709.py:3: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  print(df_idx.groupby(map_idx, axis=0).sum())


In [53]:
# dict + axis=1 (column 매핑)
df_cols = pd.DataFrame({
    "score":[80,70,90],
    "age":[20,22,21],
    "height":[170,180,160]
})
map_cols = {"score":"metrics","age":"metrics","height":"size"}
print(df_cols.groupby(map_cols, axis=1).mean())

   metrics   size
0     50.0  170.0
1     46.0  180.0
2     55.5  160.0


/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_93472/1501929358.py:8: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  print(df_cols.groupby(map_cols, axis=1).mean())


In [54]:
print(df.groupby("class", axis=0).mean(numeric_only=True))

       score   age
class             
A       80.0  21.0
B       67.5  23.5
C       84.5  24.0


/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_93472/3100137337.py:1: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  print(df.groupby("class", axis=0).mean(numeric_only=True))


In [56]:
df = pd.DataFrame({
    "class": ["A","A","A","B","B","C","C","C","C"],
    "sex":   ["F","M","F","M","F","M","F","F","M"],
    "score": [80, 70, 90, 60, 75, 88, 92, 85, 73],
    "age":   [20, 22, 21, 24, 23, 26, 22, 21, 27],
})

# MultiIndex 예제
df_mi = df.set_index(["class", "sex"]).sort_index()

df_mi

score  age
class sex            
A     F       80   20
      F       90   21
      M       70   22
B     F       75   23
      M       60   24
C     F       92   22
      F       85   21
      M       88   26
      M       73   27

In [57]:
df_mi.groupby(level=0)["score"].mean()

class
A    80.0
B    67.5
C    84.5
Name: score, dtype: float64

In [59]:
df_mi.groupby(level="sex")["score"].mean()

sex
F    84.40
M    72.75
Name: score, dtype: float64

In [61]:
df.groupby("class", as_index=True).mean(numeric_only=True)

,score,age
class,,
A,80.0,21.0
B,67.5,23.5
C,84.5,24.0


In [62]:
df.groupby("class", as_index=False).mean(numeric_only=True) # as_index=False

,class,score,age
0,A,80.0,21.0
1,B,67.5,23.5
2,C,84.5,24.0


In [ ]:
df.groupby("class", sort=True)["score"].mean()

class
A    80.0
B    67.5
C    84.5
Name: score, dtype: float64


In [64]:
print(df.groupby("class", sort=False)["score"].mean()) 

class
A    80.0
B    67.5
C    84.5
Name: score, dtype: float64


In [67]:
df_cat = df.copy()
df_cat["sex"] = pd.Categorical(df_cat["sex"], categories=["F","M","X"])  # X는 미등장

print(df_cat.groupby("sex", observed=True)["score"].mean())   # F, M

sex
F    84.40
M    72.75
Name: score, dtype: float64


In [69]:
import pandas as pd

# 샘플 데이터 생성
df = pd.DataFrame({
    "country": ["Korea", "Korea", "USA", "USA", "USA"],
    "year":    [2020, 2020, 2020, 2020, 2021],
    "value":   [30, 32, 50, 52, 55],
})

# pivot_table 사용: 국가-연도별 평균 계산
pt = df.pivot_table(
    index="country",   # 행 인덱스: 국가
    columns="year",    # 열 인덱스: 연도
    values="value",    # 집계 대상: value
    aggfunc="mean",    # 집계 함수: 평균
)
pt

year,2020,2021
country,,
Korea,31.0,NaN
USA,51.0,55.0


In [70]:
pt = df.pivot_table(
    index="country",   # 국가별
    columns="year",    # 연도별
    values="value",    # 집계 대상
    aggfunc=["sum", "count"]  # 합계와 개수
)

pt

sum       count     
year      2020  2021  2020 2021
country                        
Korea     62.0   NaN   2.0  NaN
USA      102.0  55.0   2.0  1.0

In [72]:
pt = df.pivot_table(
    index="country",
    columns="year",
    values="value",
    aggfunc="sum",      # 합계
    fill_value=0,       # 결측값 0으로 대체
    margins=True,       # 합계 행/열 추가
    margins_name="Total"
)

pt

year,2020,2021,Total
country,,,
Korea,62,0,62
USA,102,55,157
Total,164,55,219


In [73]:
df2 = pd.DataFrame({
    "country": ["Korea","Korea","USA","USA","USA"],
    "year":    [2020,2020,2020,2020,2021],
    "sales":   [100, 120, 200, 220, 300],
    "qty":     [10, 12, 20, 22, 30],
})

# sales는 합계, qty는 평균
pt = df2.pivot_table(
    index="country",
    columns="year",
    values={"sales": "sum", "qty": "mean"}
)

df2

,country,year,sales,qty
0,Korea,2020,100,10
1,Korea,2020,120,12
2,USA,2020,200,20
3,USA,2020,220,22
4,USA,2021,300,30


In [74]:
df3 = pd.DataFrame({
    "region":  ["APAC","APAC","APAC","NA","NA"],
    "country": ["Korea","Korea","Japan","USA","USA"],
    "year":    [2020,2021,2020,2020,2021],
    "value":   [1,2,3,4,5],
})

# 지역-국가를 행으로, 연도를 열로 사용
pt = df3.pivot_table(
    index=["region", "country"],  # 다중 행 인덱스
    columns=["year"],             # 열 인덱스
    values="value",
    aggfunc="sum",
    fill_value=0
)

pt

year            2020  2021
region country            
APAC   Japan       3     0
       Korea       1     2
NA     USA         4     5

In [75]:
pt_first = df.pivot_table(
	index="country", 
    columns="year", 
    values="value", 
    aggfunc="first",
)

pt_first

year,2020,2021
country,,
Korea,30.0,NaN
USA,50.0,55.0


In [76]:
pt_max   = df.pivot_table(
	index="country", 
    columns="year", 
    values="value", 
    aggfunc="max",
)

pt_max

year,2020,2021
country,,
Korea,32.0,NaN
USA,52.0,55.0


In [77]:
df2 = pd.DataFrame({
    "region": pd.Categorical(["APAC", "APAC", "NA"], categories=["APAC", "EU", "NA"]),
    "year":   pd.Categorical([2020, 2021, 2020], categories=[2020, 2021]),
    "value":  [1, 2, 3],
})

# observed=False (기본): 가능한 모든 조합 표시
pt_all = df2.pivot_table(
    index="region", columns="year",
    values="value", aggfunc="sum", fill_value=0
)
print(pt_all)  # EU도 표시됨

# observed=True: 실제로 나타난 조합만 표시
pt_obs = df2.pivot_table(
    index="region", columns="year",
    values="value", aggfunc="sum", fill_value=0, 
    observed=True,
)
print(pt_obs)  # EU 제외

year    2020  2021
region            
APAC       1     2
EU         0     0
NA         3     0
year    2020  2021
region            
APAC       1     2
NA         3     0


/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_93472/3859795884.py:8: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pt_all = df2.pivot_table(


In [78]:
# pivot_table
pt = df.pivot_table(index="country", columns="year", values="value", aggfunc="sum", fill_value=0)

# 동일한 결과를 groupby로 구현
gt = (
    df.groupby(["country", "year"])["value"]  # 국가-연도별 그룹
      .sum()                                  # 합계
      .unstack("year")                        # 열로 펼치기
      .fillna(0)                              # 결측치 0 대체
)

In [79]:
pt

year,2020,2021
country,,
Korea,62,0
USA,102,55


In [80]:
gt

year,2020,2021
country,,
Korea,62.0,0.0
USA,102.0,55.0
